## インポート

In [1]:
from config import NUMBERS, NUMBERS3, NUMBERS4, NUMBERS3_RESULT, NUMBERS4_RESULT
import sys
import re
import difflib
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib

plt.rcParams['font.family'] = 'AppleGothic'

%load_ext autoreload
%autoreload 2

##### 調査 その1

In [85]:
X = []
Y = []
Z = []
key_0 = 0
key_1 = 0
key_2 = 0
key_3 = 0
key_4 = 0
key_5 = 0
key_6  = 0 #  赤口
key_7  = 0 #  仏滅
key_8  = 0 #  友引
key_9  = 0 #  先勝
key_10 = 0 #  先負
key_11 = 0 #  大安

"""
先勝・友引・先負・仏滅・大安・赤口
"""
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time]
    _z = NUMBERS[_i][4]
    for _k, _v in enumerate(_x):
        if len(list(set(_y) & set(_v))) >= 2:
        # if len(list(set(_y) & set(_v))) >= 3 and _z == "先負":
            if _k == 0:
                key_0 += 1
            elif _k == 1:
                key_1 += 1
            elif _k == 2:
                key_2 += 1
            elif _k == 3:
                key_3 += 1
            elif _k == 4:
                key_4 += 1
            elif _k == 5:
                key_5 += 1
                
            if _z == "赤口":
                key_6 += 1
            elif _z == "仏滅":
                key_7 += 1
            elif _z == "友引":
                key_8 += 1
            elif _z == "先勝":
                key_9 += 1
            elif _z == "先負":
                key_10 += 1
            elif _z == "大安":
                key_11 += 1
            
            
            _format = "{0}回 {1}:{2}({3}) [{4}]".format(_time, _y, _x[_k], _k, _z)
            # print(_time, _y, _v)
            # print(_format)
            pass
_sum     = sum([key_0, key_1, key_2, key_3, key_4, key_5])
_format1 = "前回:{0}個, 前々回:{1}個, 前回曜日:{2}個, 前々回曜日:{3}個, 前回六曜:{4}個, 前々回六曜:{5}個".format(key_0, key_1, key_2, key_3, key_4, key_5)
_format2 = "赤口:{0}個, 仏滅:{1}個, 友引:{2}個 先勝:{3}個, 先負:{4}個, 大安:{5}個".format(key_6, key_7, key_8, key_9, key_10, key_11)

data = {
    "base": [key_0, key_1, key_2, key_3, key_4, key_5],
    "old_week": [key_6, key_7, key_8, key_9, key_10, key_11]
}
_data = pd.DataFrame(data)
print(_format1)
print(_format2)

# sns.countplot(x="old_week", data=_data)


前回:140個, 前々回:148個, 前回曜日:149個, 前々回曜日:150個, 前回六曜:143個, 前々回六曜:140個
赤口:126個, 仏滅:126個, 友引:158個 先勝:177個, 先負:134個, 大安:149個


In [102]:
# 4941
test = ["193", "413", "557", "648", "653", "669"] # 642 -> 464
# 0 -> 0
# 1 -> 2
# 2 -> 0
# 3 -> 2
# 4 -> 2
# 5 -> 3
# 6 -> 3
# 7 -> 1
# 8 -> 1
# 9 -> 2

# 4931
test = ["797", "513", "393", "909", "496", "934"] # 648 -> 480

# 4927 
test = ["393", "531", "851", "402", "934", "604"] # 496 -> 499

# 4919 
test = ["003", "402", "299", "002", "909", "372"] # 604 -> 466

"""
予測したものから、
(y, [x1, x2])
x1: 予測した数字
x2: 重複しなかった数字の合計
"""

'\n予測したものから、\n(y, [x1, x2])\nx1: 予測した数字\nx2: 重複しなかった数字の合計\n'

In [106]:
X = []
Y = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time][0]
    if len(list(set(_y) or set(_x))) == 1:
        
        print("{0} -> {1}:{2}".format(_time, _y, _x))
        # print("aaaaaaaa")
        Y.append(_y)
        X.append([_x])

4756 -> 222:276
aaaaaaaa
4611 -> 111:449
aaaaaaaa
4520 -> 111:217
aaaaaaaa
4477 -> 111:431
aaaaaaaa
4288 -> 999:784
aaaaaaaa
4127 -> 222:164
aaaaaaaa
4061 -> 222:048
aaaaaaaa
4024 -> 000:426
aaaaaaaa
4004 -> 444:799
aaaaaaaa


### 1. 前回のナンバーを特徴量とする

In [2]:
X = []
Y = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time][0]
    if len(list(set(_y) & set(_x))) >= 3:
        #print("{0} -> {1}:{2}".format(_time, _y, _x))
        Y.append(_y)
        X.append([_x])

### 2. 前回の六曜のナンバーを特徴量とする

In [33]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3[_i][_time][4]
    if len(list(set(_y) & set(_x))) >= 3:
        # print("{0} -> {1}:{2}".format(_time, _y, _x))
        Y.append(_y)
        X.append([_x])
        TIME.append(_time)

### 3. 前回のナンバーと前回の六曜のナンバーを特徴量とする

In [32]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][4]
    _x2 = NUMBERS3[_i][_time][0]
    _z = NUMBERS[_i][4]
    if len(list(set(_y) & set(_x1))) >= 2 or len(list(set(_y) & set(_x2))) >= 2:
        # Y.append(_y)
        # X.append([_x1, _x2])
        # TIME.append(_time)
        pass
    if _z == "先負":
        Y.append(_y)
        X.append([_x1, _x2])
        TIME.append(_time)

### 4. 抽選数を特徴量とする

In [47]:
X = []
Y = []
TIME = []
for _i in range(len(NUMBERS3_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x = NUMBERS3_RESULT[_i][_time]
    Y.append(_y)
    X.append(_x)
    TIME.append(_time)

#### 5. 前々回のナンバーと前回曜日ナンバーと前々回曜日のナンバーを特徴量とする

In [87]:
X    = []
Y    = []
TIME = []

"""
先勝・友引・先負・仏滅・大安・赤口
"""

for _i in range(len(NUMBERS)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x1 = NUMBERS3[_i][_time][1]
    _x2 = NUMBERS3[_i][_time][2]
    _x3 = NUMBERS3[_i][_time][3]
    _z = NUMBERS[_i][4]
    # if len(list(set(_y) & set(_x1))) >= 2 or len(list(set(_y) & set(_x2))) >= 2:
    # _z == "友引"
    # if len(list(set(_y) & set(_x1))) >= 3 or len(list(set(_y) & set(_x2))) >= 2 and _z == "友引":
        Y.append(_y)
        X.append([_x1, _x2, _x3])
        TIME.append(_time)
        pass
    #Y.append(_y)
    #X.append([_x1, _x2, _x3])
    #TIME.append(_time)

#### 予測スタート

In [88]:
y = pd.DataFrame(Y)
x = pd.DataFrame(X)

In [89]:
len(y)

34

In [90]:
model = linear_model.LinearRegression()
model.fit(X, Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [91]:
px = x
py = model.predict(px)

In [92]:
NUM = []
for _i in range(len(py)):
    _py = str(int(py[_i]))
    for _k, _v in enumerate(NUMBERS):
        if _v[0] == TIME[_i]:
            if len(list(set(_v[2]) & set(_py))) >= 2:
                print(TIME[_i], _v[2], py[_i])
                NUM.append([TIME[_i], _v[2], py[_i]])
                pass
            # print(TIME[_i], _v[2], py[_i])
print(len(NUM))

4429 724 457.57151877269473
4164 402 460.93844698943485
3997 963 569.4426432738326
3


## 決定係数

In [93]:
r2_score(y, py)

0.030786271845054447

In [94]:
model.score(x, y)

0.03078627184505445

In [95]:
model.coef_[0]

-0.00874720347904486

## 予測

In [96]:
pkl_file = "data/numbers3_20180618.pkl"

#### 前回のナンバーと前回六曜のナンバー

In [42]:
data = [float("765"), float("642")] # 20180618

##### 前々回のナンバー, 前回曜日のナンバー, 前々回の曜日ナンバー

In [97]:
data = [float("487"), float("413"), float("173")] # 20180618

##### 抽選数

In [72]:
from statistics import mean, median,variance,stdev

X0 = []
X1 = []
X2 = []
X3 = []
for _i in range(len(NUMBERS3_RESULT)):
    _time = NUMBERS[_i][0]
    _y = NUMBERS[_i][2]
    _x0 = NUMBERS3_RESULT[_i][_time][0]
    _x1 = NUMBERS3_RESULT[_i][_time][1]
    _x2 = NUMBERS3_RESULT[_i][_time][2]
    _x3 = NUMBERS3_RESULT[_i][_time][3]
    X0.append(int(_x0))
    X1.append(int(_x1))
    X2.append(int(_x2))
    X3.append(int(_x3))
    

x0 = int(mean(X0))
x1 = int(mean(X1))
x2 = int(mean(X2))
x3 = int(mean(X3))
print(x0, x1, x2, x3)

# 20180618
# data = [float("1"), float("1"), float("1"), float("1")]
data = [float("50"), float("50"), float("50"), float("50")]
# data = [float(str(x0)), float(str(x1)), float(str(x2)), float(str(x3))]

80 447 186 817


In [98]:
joblib.dump(model, pkl_file)

['data/numbers3_20180618.pkl']

In [99]:
clf = joblib.load(pkl_file)

In [100]:
processor = clf.predict([data])

In [101]:
processor

array([528.48635969])